Лабораторная работа 2. Обработка изображений и их аугментация для создания датасета.
Задание: написать программу на языке программирования Python, реализующую:

1)	Создание маски изображения (черно-белого изображения, на которой белым выделен определенный объект, который требуется сегментировать, и черным выделено все остальное) либо вручную, либо при помощи 

2)	Генератор изображений для аугментации изображения, случайным образом реализующий одну или несколько операций, таких как:
a.	Масштабирование
b.	Смещение
c.	Поворот с сохранением всего изображения
d.	Отражение по горизонтали/вертикали
e.	Применение фильтров
f.	Обрезание изображения
g.	Изменения цвета

3)	Применение реализованного генератора одновременно как к изображению, так и к маске, для получения набора данных из 100 аугментированных изображений и соответствующих им масок.

TODO: 
1. Create augmentation for images. 
2. Create better mask generator (watch video). 
3. Make english documentation for all file. 

In [13]:
import cv2 
import numpy as np 
import glob 
import os 
import random 
from scipy import ndimage
import time 
import logging
from datetime import datetime

In [14]:
# Logger for me 

log_dir = r'D:\Projects\SvetlanaDmitrievna\lab2_ii\logs'
os.makedirs(log_dir, exist_ok=True)
log_filename = os.path.join(log_dir, f'{datetime.now().strftime("%H %M %S %Y %m %d")} augmentation.log')

# Clear other handlers, started before 
logger = logging.getLogger()
for handler in logger.handlers[:]:
    logger.removeHandler(handler)

# Settings for logger 
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_filename),
        logging.StreamHandler()  # Output in console 
    ]
)
logger = logging.getLogger(__name__)

In [15]:
input_dataset = r'D:\Projects\SvetlanaDmitrievna\lab2_ii\image_dataset'

logger.info('Started collecting paths to the original images')

image_paths = []
for ext in ['*.jpg', '*.png', '*.jpeg']: 
		# os.path.join() — a function that correctly connects parts of a path in a way that 
		# takes into account the peculiarities of the operating system
		# glob.glob() — find all paths consistent with this template 
    image_paths.extend(glob.glob(os.path.join(input_dataset, ext)))

logger.info('Paths have been successfully assembled')
logger.info(f"{len(image_paths)} images in input dataset")

output_dataset = r'D:\Projects\SvetlanaDmitrievna\lab2_ii\masks_dataset'

# def mask_generator(image_paths): 
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     # Последнее значение — значение кол-ва ядер. Чем больше, тем больше можно сделать размытие. 
#     blurred = cv2.GaussianBlur(gray, (25,25), 10)

#     _, mask = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY)

#     # kernel — ядро для морфологических операций, создаёт матрицу размером 21 на 21.
#     # Она используется, как шаблон для обработки изображения в cv2.morphologyEx
#     kernel = np.ones((21,21))  

#     # Выполняет морфологические преобразованяи на бинарном изображении 
#     mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

#     output_path = os.path.join(output_dataset, f'mask{i+1}.png')

#     cv2.imwrite(output_path, mask)

def improved_mask_generator(image_paths):
	"""
	How it works? 

	Blur → easier to outline 
	Canny edge detector → 
	Dilate → 
	Find contours → 
	Draw white within contours → 
	Morph close → 
	Morph open
	"""

	for i, image in enumerate(image_paths):
		image = cv2.imread(image)

		gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
		# The last value is the value of the number of kernels. 
		# The more you have kernels, the more blurring you can do. 
		# Larger kernel size create more blur. E.g. (5, 5) → (21, 21) more blur. 
		blurred = cv2.GaussianBlur(gray, (5,5), 0)

		# Canny edge detector 
		# 60 — lower threshold: if smaller the value, 
		# the more faint edges will be detected.
		# 120 — upper threshold: if higher the value, 
		# the fewer edges will be included in the final result.
		edges = cv2.Canny(blurred, 60, 120) # initial values — 50 120 

		# Expands areas of white pixels in a binary image.
		# np.ones((3,3)) — kernel 
		# iterations — the number of times the operation will be repeated, 
		# the higher the number of iterations the edges will be widened → more white color. 
		dilated = cv2.dilate(edges, np.ones((3, 3)), iterations=2)  

		# cv2.RETR_EXTERNAL — means “retrieve only external contours”:
		# The function finds only contours that are not inside other contours. (explanation in obsidian)
		#
		# cv2.CHAIN_APPROX_SIMPLE — simplifies contours by preserving only their key points.
		# For example, for a straight line, only two endpoints are preserved.
		# For a rectangle, only 4 vertices are saved instead of all perimeter points. 
		# This saves memory and facilitates further processing of the contours.
		#
		# `contours` — list of found contours, where each contour is represented 
		# by an array of points (x, y)
		contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
		mask = np.zeros_like(gray)

		# drawContours fills in all the detected contours with white color. 
		#
		# `mask` — The destination image where contours will be drawn 
		# (a blank black image created with np.zeros_like(gray))
		# First `-1` — when set to -1, it means "draw all contours". 
		# `255` — color to fill contours (in grayscale, if value smaller, color be more black)
		# Second `-1`— the thickness of the contour line.  When set to -1, it means "fill the contour" 
		# rather than just drawing its outline.
		cv2.drawContours(mask, contours, -1, 255, -1)  

		# kernel - kernel for morphology operations, creates a 21 by 21 matrix.
		# It is used as a template for image processing in cv2.morphologyEx. 
		kernel = np.ones((27,27), np.uint8)  

		# Performs morphological transformations on a binary image
		# 
		# MORPH_CLOSE: Dilation → Erosion
		# 1. Dilation (dilation of the white areas)
		# 2. Erosion (narrowing of the white areas) 
		# Simply put, it fills small black holes inside white areas, 
		# connects closely spaced areas, and smooths the contours of objects. 
		#
		# MORPH_OPEN: Reversed sequence. 
		# 1. Erosion (narrowing of white areas)
		# 2. Dilation (dilation of white areas)
		#
		# Simply put, removes small white noise and protrusions, separates thin connections between objects,
		# removes thin protrusions from a contour, smooths object contours.
		mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
		mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

		output_path = os.path.join(output_dataset, f'mask{i+1}.png')

		cv2.imwrite(output_path, mask)

logger.info('Generation of source image masks started')

# Makes masks
improved_mask_generator(image_paths)

logger.info('Masks generation has been successfully completed')

output_paths = glob.glob(os.path.join(output_dataset, '*.png'))

logger.info(f"{len(output_paths)} masks in masks dataset")


2025-03-12 11:57:23,799 - INFO - Started collecting paths to the original images
2025-03-12 11:57:23,801 - INFO - Paths have been successfully assembled
2025-03-12 11:57:23,802 - INFO - 10 images in input dataset
2025-03-12 11:57:23,803 - INFO - Generation of source image masks started
2025-03-12 11:57:23,943 - INFO - Masks generation has been successfully completed
2025-03-12 11:57:23,944 - INFO - 10 masks in masks dataset


In [16]:
# Functions for aumentation 

def scale_image(image, scale_factor=0.7):
	"""
	Mask scaling.
	image = cv2.imread(image_path). 
	""" 

	height, width = image.shape[:2]
	new_height, new_width = int(height * scale_factor), int(width * scale_factor)

	if scale_factor >= 1.0: 
		# For upscaling use INTER_LANCZOS4, because it gives best quality for upscaling. 
		interpolation=cv2.INTER_LANCZOS4
	else: 
		# For downscaling use INTER_AREA, this is a pixel area interpolation method:
		# prevents distortion when shrinking, preserves details better, when reducing expansion, 
		# gives better results than other methods. 
		interpolation=cv2.INTER_AREA
		
	return cv2.resize(image, (new_width, new_height), interpolation=interpolation)        


def shift_image(image, shift_limit=0.1): 
	"""Mask shift"""
	height, width = image.shape[:2]
	tx = random.uniform(-shift_limit, shift_limit) * width
	ty = random.uniform(-shift_limit, shift_limit) * height

	# [1, 0, tx], [0, 1, ty] — matrix`s strings
	shift_matrix = np.float32([[1, 0, tx], [0, 1, ty]]) 

	# Affine transformation matrix
	# x + tx (horizontal shift)
	# y + ty (vertical shift)
	return cv2.warpAffine(image, shift_matrix, (width, height))


def rotate_image(image, angle): 
	"""Rotate image without loss information"""
	height, width = image.shape[:2]

	diagonal = np.sqrt(height**2 + width**2)
	# The new size is int(np.ceil(diagonal)) because
	# the maximum space that can be required
	# when rotating a rectangle by an arbitrary angle is determined 
	# by the length of its diagonal.
	# 
	# Why int()? because OpenCV requires integer values.
	# And an extra pixel ensures that everything fits on the screen.
	#
	# For example: 
	# You have image 100x200 pixels. Diagonal equals ~223.6. New_size = 224. 
	# 
	# Canvas 224x224 guaranteed to accommodate image 100x200, when its rotated to any angle. 
	new_size = int(np.ceil(diagonal))

	# x_center & y_center are used to find 
	# the coordinates of the upper left corner, 
	# where you want to place the original image → that it's exactly centered on the new canvas.
	x_center = (new_size - width) // 2 
	y_center = (new_size - height) // 2 

	# For colored image: 
	# image.shape → (height, width, channels) 
	# channels — quantity of color channels: 3 for RGB, 4 for RGBA|BGRA (alpha is transparency channel)
	#
	# For black & white iamge: 
	# image.shape → (height, width)
	if len(image.shape) == 3: 

		# np.zeros — array with zeros 
		# (new_size, new_size, 3) — amount strings, amount columnsm, amount channels 
		# dtype=np.uint8 — 8-bit unsigned integer in [0, 255]
		# 
		# What means '3'? It means that simple zero to be [0, 0, 0] 
		# like rgb code 
		# 
		# For example, matrix 2x3 (strings x columns): 
		# Pixel [0, 0]; [0, 0, 0]    Pixel [0, 1]; [0, 0, 0]     Pixel [0, 2]; [0, 0, 0]
		# Pixel [1, 0]; [0, 0, 0]    Pixel [1, 1]; [0, 0, 0]     Pixel [1, 2]; [0, 0, 0]
		canvas = np.zeros((new_size, new_size, 3), dtype=np.uint8)
	else:
		# here is for black & white images
		canvas = np.zeros((new_size, new_size), dtype=np.uint8)

	# This string copy source image to new and bigger canvas in 
	# concrete position.
	# For vertical axis it left bottom corner from source image to left top corner. 
	# For horizontal axis it left bottom corner to right bottom corner for source image.
	canvas[y_center:y_center+height, x_center:x_center+width] = image 

	# Creates an affine transformation matrix to rotate image 
	# Parameters: 
	# 1. center — pivot point in the format (x, y) 
	# 2. angle 
	# 3. scale — scale factor 
	rot_matrix = cv2.getRotationMatrix2D((new_size/2, new_size/2), angle, 1.0)

	# Apply Affine transformation with given rot_matrix.
	# Parameters: 
	# 1. src: canvas — source image to which the transformation will be applied.
	# 2. M: rot_matrix — Affine transformation matrix. 
	# 3. dsize: (new_size, new_size) — size of source image 
	rotated_image = cv2.warpAffine(canvas, rot_matrix, (new_size, new_size))

	return rotated_image

def flip_image(image, mode): 
	"""
	Flips image. 
	
	Mods: 
	0 - vertical flip; 
	1 - horizontal flip; 
	-1 - both axis; 
	"""

	return cv2.flip(image, mode)


def apply_filter(image, filter_type='sharpen'): 
	"""
	Applies different filters to image. 
	
	Filters: 
	'blur' — blurring 
	'sharpen' — sharpenning filter
	'canny_edge' — for detecting edges of objects in a picture 
	'emboss' — emboss 3d effect 
	"""
	match filter_type: 
		case 'blur':
			# (21, 21) — kernel size (width, height).
			#  
			# The features of this parameter are: 
			# 1. The size must be positive and odd (3, 5, 7, ...) 
			# 2. larger size creates more blur. 
			# 3. (21,21) means that a 21×21 pixel area will be used to calculate 
			# the new value of each pixel. 
			# 4. square kernels are used (same width and height value), because they 
			# affect equally in all directions, symmetry simplifies computations, 
			# it is easier to implement a square window, it is easier to handle edge 
			# cases, many libraries are better optimized to work with square kernels, 
			# so historically speaking. 
			# 
			# Parametr '5' — sigma, standard deviation. 
			# 
			# Рow sigma affects the result?
			# 1. Small sigma value (E.g., 0.5): sharper transition between pixels, less blurring. 
			# 2. Big sigma value (E.g., 5.0): smoother transition, stronger blurring. 
			return cv2.GaussianBlur(image, (21,21), 5)
	
		case 'sharpen': 
			# kernel — matrix 3x3 with cenеral '9' and other '-1'
			# Operating principle:
			# cenеral '9' increase the weight of the current pixel
			# other '-1' decrease part of values from nearest pixels
			# → this results in stronger edges and details. 
			kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])

			# '-1' means image depth will be the same as the source image 
			# With the convolution operation, the kernel “walks” through all pixels 
			# and calculates new values for them: 
			# The current pixel is taken as the center pixel, 
			# and its new value is found as the sum of the multiplied kernel values 
			# of the nearest pixels + the product of the current pixel with the center 
			# value in the kernel. 
			#
			# Use cv2.BORDER_CONSTANT, because i want to be complemented 
			# on the edges of 0 → smaller sharpness in edges, i hope 
			return cv2.filter2D(image, -1, kernel, borderType=cv2.BORDER_CONSTANT)
	
		case 'canny_edge': 

			# Canny Edge Detector — algorithm for detecting edges of objects in a picture
			# Makes binary image, where white pixels (255) are edges of objects, black (0) — other. 
			# 100 — lower threshold (if smaller the value, 
			# the more faint edges will be detected.)
			# 200 — upper threshold (if higher the value, 
			# the fewer edges will be included in the final result.)
			return cv2.Canny(image, 140, 200) # 100 200
		
		case 'emboss': 

			# Negative values (-2, -1) in the upper left corner create shadows 
			# Positive values (1, 2) in the lower right corner create illuminated areas
			# Gradient transition from negative to positive values 
			# creates a 3d effect
			kernel = np.array([
				[-2, -1, 0], 
				[-1, 1, 1],
				[0, 1, 2]
			])

			# cv2.BORDER_REFLECT — when the kernel goes beyond the image boundary, 
			# the algorithm reflects (mirrors) the pixels away from the boundary
			emboss_image = cv2.filter2D(image, -1, kernel, borderType=cv2.BORDER_REFLECT)
			return emboss_image
	
		case _: 
			return image 
	

def crop_image(image, crop_factor=0.81): 
	"""
	Croping image.
	"""
	height, width = image.shape[:2]

	crop_height = int(height * crop_factor)
	crop_width = int(width * crop_factor)

	# By random.randint randomly select the crop area
	x = random.randint(0, width - crop_width)
	y = random.randint(0, height - crop_height)

	return image[y:y+crop_height, x:x+crop_width]


def change_color(image, mode='hsv'): 
	"""
	Changing color. 
	'hsv_shift' — changing hue 
	'brightness' — changing brightness 
	'contrast' — changing picture contrast 
	'saturation' — changing saturation
	"""

	if len(image.shape) < 3: 
		return image 
	
	match mode:
		case 'hsv_shift': 
			# cvtColor — func, which transform image from one color space to other
			# RGB → HSV (what is HSV? in obsidian)
			# 
			# HSV:
			# 1. Hue — [0, 180]
			# 2. Saturation — [0, 255]
			# 3. Value — [0, 255]
			hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

			# Split hsv image to 3 single-channels images. 
			h, s, v = cv2.split(hsv)

			# Append to hue random number to create new hue. 
			h = (h + random.randint(0, 180)) % 180 

			# combines several single-channel images into a single multi-channel image.
			hsv = cv2.merge([h, s, v])
			return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

		case 'brightness': 

			hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
			h, s, v = cv2.split(hsv)

			# factor < 1 → image is darker 
			# factor > 1 → image is brighter  
			factor = random.uniform(0.7, 1.3)

			# np.clip cutting values that smaller then 0 or bigger then 255
			# smaller 0 → value sets as 0 
			# bigger 255 → sets as 255 
			# 
			# astype transforms np.clip() output to 8-bit insigned int
			# othwerwise,the number would still be a float, 
			# which openCV does not handle correctly. 
			v = np.clip(v * factor, 0, 255).astype(np.uint8)
			hsv = cv2.merge([h, s, v])
			return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
		
		case 'contrast': 
			# factor > 1 → image more contrast
			# factro < 1 → image less contrast 
			factor = random.uniform(0.1, 4)

			# for colorful image `image` is three-dimensional array: [height, width, 3]
			# where `3` is number of channels RED-GREEN-BLUE 
			# 
			# How it calculate mean number from image? 
			# E.g. we have 100x100 image with 3 channels → 100 x 100 x 3 = 30000 
			# We take pixel, sum all values of pixel from 3 channels → sum([R, G, B])
			# That's how all the pixels are counted. 
			# Then this sum divided by 30000 (`height * width * 3`)
			mean = np.mean(image)

			# This formul `(image - mean) * factor + mean` means: 
			# 1. dark pixels will be darker 
			# 2. bright pixels will be brighter 
			# 3. pixels equals mean will remain unchanged. 
			#
			# How rgb works? 
			# Each color channel has an intensity:
			# ranging from zero (complete darkness) to 255 (maximum brightness)
			return np.clip((image - mean) * factor + mean, 0, 255).astype(np.uint8)

		case 'saturation': 
			hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
			h, s, v = cv2.split(hsv)

			factor = random.uniform(0.3, 2)

			s = np.clip(s * factor, 0, 255).astype(np.uint8)

			hsv = cv2.merge([h, s, v])
			return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
		
		case _: 
			return image  
		

In [17]:
# Folder for the augmented images and augmented maskes
augmented_images = r'D:\Projects\SvetlanaDmitrievna\lab2_ii\augmented_images'
augmented_masks = r'D:\Projects\SvetlanaDmitrievna\lab2_ii\augmented_masks'

flip_mods = [0, 1, -1]
filter_types = ['blur', 'sharpen', 'canny_edge', 'emboss']
color_mods = ['hsv_shift', 'brightness', 'contrast', 'saturation'] 

# Create augmentations for source images
def images_augmentation(image_paths):
	for i, image in enumerate(image_paths):

		scale_factor_range = (random.uniform(0.3, 0.7), random.uniform(1.2, 1.5))
		# returns float in (for example) [0.7, 1.3] → e.g.: 0.87, 1.11, 0.9 and e.t.c
		scale_factor = random.uniform(scale_factor_range[0], scale_factor_range[1])
		shift_limit = random.uniform(0.01, 0.4)
		angle = random.uniform(-180, 180)
		flip_mode = random.choice(flip_mods)
		filter_type = random.choice(filter_types)
		crop_factor = random.uniform(0.69, 0.99)
		color_mode = random.choice(color_mods)

		image = cv2.imread(image)
		new_image = scale_image(image, scale_factor=scale_factor)
		new_image = shift_image(new_image, shift_limit=shift_limit)
		new_image = rotate_image(new_image, angle=angle)
		new_image = flip_image(new_image, mode=flip_mode)
		new_image = apply_filter(new_image, filter_type=filter_type)
		new_image = crop_image(new_image, crop_factor=crop_factor)
		new_image = change_color(new_image, mode=color_mode)

		# Logging all proccess 
		logger.info(f'For Image{i+1} augmentation proccess starting!')
		logger.info(f'Scale Image{i+1} with scale_factor={scale_factor}')
		logger.info(f'Shift Image{i+1} with shift_limit={shift_limit}')
		logger.info(f'Rotate Image{i+1} with angle={angle}')
		logger.info(f'Flip Image{i+1} with flip_mode={flip_mode}')
		logger.info(f'Applied filter for Image{i+1} with filter_type={filter_type}')
		logger.info(f'Crop Image{i+1} with crop_factor={crop_factor}')
		logger.info(f'Change Image{i+1} color with color_mode={color_mode}')
		logger.info(f'Image{i+1} is successfully augmentated!')

		output_path = os.path.join(augmented_images, f'augment_image{i+1}.png')
		cv2.imwrite(output_path, new_image)

# Make augmentations for masks 
def masks_augmentation(output_paths):
	for i, mask in enumerate(output_paths): 

		scale_factor_range = (random.uniform(0.3, 0.7), random.uniform(1.2, 1.5))
		# returns float in (for example) [0.7, 1.3] → e.g.: 0.87, 1.11, 0.9 and e.t.c
		scale_factor = random.uniform(scale_factor_range[0], scale_factor_range[1])
		shift_limit = random.uniform(0.01, 0.4)
		angle = random.uniform(-180, 180)
		flip_mode = random.choice(flip_mods)
		filter_type = random.choice(filter_types)
		crop_factor = random.uniform(0.69, 0.99)
		color_mode = random.choice(color_mods)

		mask = cv2.imread(mask)
		new_mask = scale_image(mask, scale_factor=scale_factor)
		new_mask = shift_image(new_mask, shift_limit=shift_limit)
		new_mask = rotate_image(new_mask, angle=angle)
		new_mask = flip_image(new_mask, mode=flip_mode)
		new_mask = apply_filter(new_mask, filter_type=filter_type)
		new_mask = crop_image(new_mask, crop_factor=crop_factor)
		new_mask = change_color(new_mask, mode=color_mode)

		# Logging all proccess 
		logger.info(f'For Mask{i+1} augmentation proccess starting!')
		logger.info(f'Scale Mask{i+1} with scale_factor={scale_factor}')
		logger.info(f'Shift Mask{i+1} with shift_limit={shift_limit}')
		logger.info(f'Rotate Mask{i+1} with angle={angle}')
		logger.info(f'Flip Mask{i+1} with flip_mode={flip_mode}')
		logger.info(f'Applied filter for Mask{i+1} with filter_type={filter_type}')
		logger.info(f'Crop Mask{i+1} with crop_factor={crop_factor}')
		logger.info(f'Change Mask{i+1} color with color_mode={color_mode}')
		logger.info(f'Mask{i+1} is successfully augmentated!')
		
		output_path = os.path.join(augmented_masks, f'augment_mask{i+1}.png')
		cv2.imwrite(output_path, new_mask)

logger.info('Started images augmention process') 
images_augmentation(image_paths)
logger.info('Images augmention process successfully done!') 

logger.info('Started mask augmentation process')
masks_augmentation(output_paths)
logger.info('Masks augmention process successfully done!') 



2025-03-12 11:57:23,973 - INFO - Started images augmention process
2025-03-12 11:57:24,000 - INFO - For Image1 augmentation proccess starting!
2025-03-12 11:57:24,000 - INFO - Scale Image1 with scale_factor=0.6635296423022164
2025-03-12 11:57:24,001 - INFO - Shift Image1 with shift_limit=0.33060213646595593
2025-03-12 11:57:24,003 - INFO - Rotate Image1 with angle=32.796080485485504
2025-03-12 11:57:24,004 - INFO - Flip Image1 with flip_mode=-1
2025-03-12 11:57:24,005 - INFO - Applied filter for Image1 with filter_type=sharpen
2025-03-12 11:57:24,006 - INFO - Crop Image1 with crop_factor=0.6925596933168687
2025-03-12 11:57:24,007 - INFO - Change Image1 color with color_mode=contrast
2025-03-12 11:57:24,008 - INFO - Image1 is successfully augmentated!
2025-03-12 11:57:24,028 - INFO - For Image2 augmentation proccess starting!
2025-03-12 11:57:24,029 - INFO - Scale Image2 with scale_factor=1.2643374796083144
2025-03-12 11:57:24,029 - INFO - Shift Image2 with shift_limit=0.322604700774566